In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import nltk
import string
import re
from os import path
from PIL import Image

from spellchecker import SpellChecker
from nltk import FreqDist
from collections import Counter
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(strip_handles=True)



In [2]:
stop_words = stopwords.words('english')

In [3]:
df_train = pd.read_csv("dataset/train.csv")
df_test= pd.read_csv("dataset/test.csv")
pd.set_option('display.max_colwidth', -1) #elimino limite de truncado para visualizacion

/home/pablo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# contador de palabras
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# palabras unicas
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))

# contador de stopwords
df_train['stop_word_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
df_test['stop_word_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

# contador de url
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# extractor de emails
df_train['emails'] = df_train['text'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9+._-]+\.[a-zA-Z0-9+._-]+)', x))
df_test['emails'] = df_test['text'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9+._-]+\.[a-zA-Z0-9+._-]+)', x))

# contador de emails
df_train['emails_count'] = df_train['emails'].apply(lambda x: len(x))
df_test['emails_count'] = df_test['emails'].apply(lambda x: len(x))

# promedio del largo de palabras
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# contador de caracteres
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# contador de numeros
df_train['number_count'] = df_train['text'].apply(lambda x: len([w for w in x.split() if w.isdigit()]))
df_test['number_count'] = df_test['text'].apply(lambda x: len([w for w in x.split() if w.isdigit()]))

# contador de mayuscuslas en palabras 
df_train['upper_count'] = df_train['text'].apply(lambda x: len([w for w in x.split() if w.isupper() and len(x)>3]))
df_test['upper_count'] = df_test['text'].apply(lambda x: len([w for w in x.split() if w.isupper() and len(x)>3]))

# contador de puntuaciones
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# contador de hashtags
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# contador de menciones
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [5]:
# con el fin de agilizar el tiempo para la construccion de algunos filtros se han recolectado de diferentes fuentes

contracciones = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "i would",
  "i'd've": "i would have",
  "i'll": "i will",
  "i'll've": "I will have",
  "i'm": "i am",
  "i've": "i have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}
    
def convertir_contracciones(text):
    t=[]
    words=text.split()
    t = [contracciones[w.lower()] if w.lower() in contracciones.keys() else w for w in words]
    return ' '.join(t) 

df_train['text']=df_train['text'].apply(lambda x: convertir_contracciones(x))
df_test['text']=df_test['text'].apply(lambda x: convertir_contracciones(x))



def limpieza(text):
            
    # elimino caracteres especiales
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    # Reemplazo contracciones
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"We're", "We are", text)
    text = re.sub(r"That's", "That is", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"they're", "they are", text)
    text = re.sub(r"Can't", "Cannot", text)
    text = re.sub(r"wasn't", "was not", text)
    text = re.sub(r"don\x89Ûªt", "do not", text)
    text = re.sub(r"aren't", "are not", text)
    text = re.sub(r"isn't", "is not", text)
    text = re.sub(r"What's", "What is", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r"hasn't", "has not", text)
    text = re.sub(r"There's", "There is", text)
    text = re.sub(r"He's", "He is", text)
    text = re.sub(r"It's", "It is", text)
    text = re.sub(r"You're", "You are", text)
    text = re.sub(r"I'M", "I am", text)
    text = re.sub(r"shouldn't", "should not", text)
    text = re.sub(r"wouldn't", "would not", text)
    text = re.sub(r"i'm", "I am", text)
    text = re.sub(r"I\x89Ûªm", "I am", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r"Isn't", "is not", text)
    text = re.sub(r"Here's", "Here is", text)
    text = re.sub(r"you've", "you have", text)
    text = re.sub(r"you\x89Ûªve", "you have", text)
    text = re.sub(r"we're", "we are", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"we've", "we have", text)
    text = re.sub(r"it\x89Ûªs", "it is", text)
    text = re.sub(r"doesn\x89Ûªt", "does not", text)
    text = re.sub(r"It\x89Ûªs", "It is", text)
    text = re.sub(r"Here\x89Ûªs", "Here is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"I\x89Ûªve", "I have", text)
    text = re.sub(r"y'all", "you all", text)
    text = re.sub(r"can\x89Ûªt", "cannot", text)
    text = re.sub(r"would've", "would have", text)
    text = re.sub(r"it'll", "it will", text)
    text = re.sub(r"we'll", "we will", text)
    text = re.sub(r"wouldn\x89Ûªt", "would not", text)
    text = re.sub(r"We've", "We have", text)
    text = re.sub(r"he'll", "he will", text)
    text = re.sub(r"Y'all", "You all", text)
    text = re.sub(r"Weren't", "Were not", text)
    text = re.sub(r"Didn't", "Did not", text)
    text = re.sub(r"they'll", "they will", text)
    text = re.sub(r"they'd", "they would", text)
    text = re.sub(r"DON'T", "DO NOT", text)
    text = re.sub(r"That\x89Ûªs", "That is", text)
    text = re.sub(r"they've", "they have", text)
    text = re.sub(r"i'd", "I would", text)
    text = re.sub(r"should've", "should have", text)
    text = re.sub(r"You\x89Ûªre", "You are", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"Don\x89Ûªt", "Do not", text)
    text = re.sub(r"we'd", "we would", text)
    text = re.sub(r"i'll", "I will", text)
    text = re.sub(r"weren't", "were not", text)
    text = re.sub(r"They're", "They are", text)
    text = re.sub(r"Can\x89Ûªt", "Cannot", text)
    text = re.sub(r"you\x89Ûªll", "you will", text)
    text = re.sub(r"I\x89Ûªd", "I would", text)
    text = re.sub(r"let's", "let us", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"you're", "you are", text)
    text = re.sub(r"i've", "I have", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"i'll", "I will", text)
    text = re.sub(r"doesn't", "does not", text)
    text = re.sub(r"i'd", "I would", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"ain't", "am not", text)
    text = re.sub(r"you'll", "you will", text)
    text = re.sub(r"I've", "I have", text)
    text = re.sub(r"Don't", "do not", text)
    text = re.sub(r"I'll", "I will", text)
    text = re.sub(r"I'd", "I would", text)
    text = re.sub(r"Let's", "Let us", text)
    text = re.sub(r"you'd", "You would", text)
    text = re.sub(r"It's", "It is", text)
    text = re.sub(r"Ain't", "am not", text)
    text = re.sub(r"Haven't", "Have not", text)
    text = re.sub(r"Could've", "Could have", text)
    text = re.sub(r"youve", "you have", text)  
    text = re.sub(r"donå«t", "do not", text)    
    
    return text


df_train['text']=df_train['text'].apply(lambda x: limpieza(x))
df_test['text']=df_test['text'].apply(lambda x: limpieza(x))

In [6]:
def eliminacion(text):
    # elimino Retweet RT
    text = re.sub(r'RT', "", text)
    
    # elimino urls
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text)
    
    # elimino puntuaciones
    text = re.sub('[^A-Z a-z 0-9-]+', '', text)
    
    # elimino espacios dobles
    text = re.sub(r"\s+", " ", text)
    
    # convierto a minisculas
    text = text.lower()
    
    
    return text


df_train['text']=df_train['text'].apply(lambda x: eliminacion(x))
df_test['text']=df_test['text'].apply(lambda x: eliminacion(x))

In [7]:
#corrector de palabras
spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

df_train['text']=df_train['text'].apply(lambda x : correct_spellings(x))
df_test['text']=df_test['text'].apply(lambda x : correct_spellings(x))

In [8]:
df_train.to_csv('dataset/trainSpellChecker.csv', index=False)
df_test.to_csv('dataset/testSpellChecker.csv', index=False)

In [10]:
df_train.loc[4545:4550]

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,url_count,emails,emails_count,mean_word_length,char_count,number_count,upper_count,punctuation_count,hashtag_count,mention_count
4545,6462,injured,Doghouse,injured barcelona full-back alba out of super cup,0,9,9,2,1,[],0,7.111111,72,0,0,6,0,0
4546,6463,injured,NaN,michaelgbaron how come scott rice does not get another shot holding lefties to a 184 average is he injured,0,18,18,6,0,[],0,5.166667,110,0,0,6,0,1
4547,6464,injured,Mumbai,udhampur terror attack militants attack police post 2 spos injured suspected militants tonight attacked a p,1,17,17,1,1,[],0,7.000000,135,1,0,10,0,0
4548,6465,injured,someplace living my life,so bautista injured wilshire,0,4,4,1,0,[],0,6.250000,28,0,0,0,0,0
4549,6466,injured,USA,offers 8392 deluxe toilet safety supporthealthhomebathroomsupportelderlyinjureds,1,9,9,0,2,[],0,14.555556,139,0,0,20,1,0
4550,6467,injured,NaN,washington post - 4 dead dozens injured in gaza blast near house levelled in summer war,1,17,16,2,1,[],0,5.470588,109,1,0,6,0,0


In [12]:
df_train.loc[4545:4550]

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,url_count,emails,emails_count,mean_word_length,char_count,number_count,upper_count,punctuation_count,hashtag_count,mention_count
4545,6462,injured,Doghouse,injured barcelona full-back alba out of super cup,0,9,9,2,1,[],0,7.111111,72,0,0,6,0,0
4546,6463,injured,NaN,michaelgbaron how come scott rice does not get another shot holding lefties to a 184 average is he injured,0,18,18,6,0,[],0,5.166667,110,0,0,6,0,1
4547,6464,injured,Mumbai,udhampur terror attack militants attack police post 2 spos injured suspected militants tonight attacked a p,1,17,17,1,1,[],0,7.000000,135,1,0,10,0,0
4548,6465,injured,someplace living my life,so bautista injured wilshire,0,4,4,1,0,[],0,6.250000,28,0,0,0,0,0
4549,6466,injured,USA,offers 8392 deluxe toilet safety supporthealthhomebathroomsupportelderlyinjureds,1,9,9,0,2,[],0,14.555556,139,0,0,20,1,0
4550,6467,injured,NaN,washington post - 4 dead dozens injured in gaza blast near house levelled in summer war,1,17,16,2,1,[],0,5.470588,109,1,0,6,0,0
